In [1]:
from random import randint
from time import time 
import requests
from requests import get
from warnings import warn 
from bs4 import BeautifulSoup
from IPython.core.display import clear_output
from time import sleep
import csv 

In [ ]:
# set errors to track number of erros
errors = 0
# track time during web scraping
start_time = time()
# track number of requests
requests = 0
# go through pages of craigslist rentals, divided 120 each page
# and seemingly 2400 or less listings in total for the default Minneapolis
# craigslist page
listings_pages = [str(i) for i in range(0, 2400) if (i%120==0)]
# iterate through listing pages which 
# contain numbers between 0-2400 divisble by 120
for page in listings_pages:
    # for page increment value by 120 each time to loop through 
    # groupings size 120 of craigslist rental ads and make request to url plus
    # grouping indicated by number divisible by 120
    driver = get('https://minneapolis.craigslist.org/search/apa?s=' + (page))
    # create beautiful soup object
    listing_object = BeautifulSoup(driver.content, 'html.parser')
    # listing containers contains all 120 urls from the url requested with driver variable
    listing_containers = listing_object.find_all('li', class_="result-row")
    # for each individual url contained in listing_containers
    for item in listing_containers:
        # increment requests for tracking purposes
        requests += 1 
        # track time
        elapsed_time = time() - start_time
        print ('Request: {}; Frequency: {} request/s'.format(requests, requests/elapsed_time))
        # clear output for continous display of values
        clear_output(wait = True)
        # stop loop at 2500 requests
        if requests > 2500:
            warn('Number of requests was greater than 2280')
            break
        # get url of each link that appears on the page of 120 listings
        try:
            url = item.find('a', class_="result-title hdrlnk")['href']
        except AttributeError:
            print ('cant append')
        try:
            # make request to url leading to single rental listing
            link = get(url)
            # create Beatufiul Soup object
            listing_object = BeautifulSoup(link.content, 'html.parser')
            # find everything in the "section tag of each url
            page_items = listing_object.find_all('section', class_="page-container")
            # scrape desired attributes by looping through 
            # "section" tag of each url
            for container in page_items:
                # these lists will become the columns for the data frame 
                # columns that they become are commented above each respective list
                # the url list is not here but the variable is already set during each iteration as the 
                # 'price'
                prices = []
                # 'beds_baths'
                bed_baths = []
                # 'address'
                addrs = []
                # 'latitude'
                lats = []
                # 'longitude'
                lons = []
                # 'map accuracy'
                accuracy = []
                # 'datetime'
                date_times = []
                # 'title'
                data_titles = []
                # 'post id'
                posting_ids = []
                # 'square feet'
                areas = []
                # 'square feet 2'
                title_price_and_attributes = []
                # get price
                try:
                    price = container.h2.find("span", class_="price").text
                    prices.append(price)
                except AttributeError:
                    prices.append("N/A")
                except TypeError:
                    prices.append("N/A")
                # get latitutude
                try:
                    lat = container.find('div', class_="viewposting")['data-latitude']
                    lats.append(lat)
                except TypeError:
                    lats.append("N/A")
                except AttributeError:
                    lats.append("N/A")
                # get longitude
                try:
                    lon = container.find('div', class_="viewposting")['data-longitude']
                    lons.append(lon)
                except TypeError:
                    lons.append("N/A")
                except AttributeError:
                    lons.append("N/A")
                # get map data accuracy
                try:
                    data_accuracy = container.find('div', class_="viewposting")['data-accuracy']
                    accuracy.append(data_accuracy)
                except TypeError:
                    accuracy.append("N/A")
                except AttributeError:
                    accuracy.append("N/A")
                # get datetime
                try:
                    date_time = container.find('time', class_="date timeago")['datetime']
                    date_times.append(date_time)
                except TypeError:
                    date_times.append("N/A")
                except AttributeError:
                    date_times.append("N/A")
                # get bedrooms and baths
                try:
                    bed_bath = container.section.find_all("span", class_="shared-line-bubble")[0].text
                    bed_baths.append(bed_bath)
                except AttributeError:
                    bed_baths.append("N/A")
                except TypeError:
                    bed_baths.append("N/A")
                except IndexError:
                    bed_baths.append("N/A")
                # get square feet
                try: 
                    sqrft = container.section.find_all("span", class_="shared-line-bubble")[1].text
                    areas.append(sqrft)
                except AttributeError:
                    areas.append("N/A")
                except TypeError:
                    areas.append("N/A")
                except IndexError:
                    areas.append("N/A")
                # get address 
                try:   
                    addr = container.section.section.find("div", class_="mapaddress").text
                    addrs.append(addr)
                except AttributeError:
                    addrs.append("N/A")
                # get title
                try:
                    data_title = container.h2.find(id="titletextonly").text
                    data_titles.append(data_title)
                except AttributeError:
                    data_titles.append("N/A")
                # get posting id
                try:
                    posting_id = container.find_all('p', class_="postinginfo")[1].text
                    posting_ids.append(posting_id)
                except AttributeError:
                    posting_ids.append("N/A")
                except IndexError:
                    posting_ids.append("N/A")
                # this will be information in square feet 2
                try:
                    price_and_attributes = container.h2.find('span', class_="housing").text
                    title_price_and_attributes.append(price_and_attributes)
                except AttributeError:
                    title_price_and_attributes.append("N/A")
                # dataframe is updated in each loop 
                apartment_data = pd.DataFrame({'Url' : url, 
                                           'Price': prices,
                                           'Address': addrs,
                                           'Latitude': lats,
                                           'Longitude': lons,
                                           'Map Accuracy': accuracy,
                                           'Date & Time Stamp': date_times,
                                           'Bath/Bed': bed_baths, 
                                           'Areas' : areas,
                                           'Listing Title': data_titles,
                                           'ids' : posting_ids,
                                           'Header': title_price_and_attributes
                                           })
                # leave dataframe open and call "a" for append option to append values iteratively
                with open("test.csv", "a", encoding='utf-8') as f:
                    apartment_data.to_csv(f, header=False)
        # keep track of requests that come back errors
        except:
            errors += 1
            print(errors)
            continue  

120
Request: 121; Frequency: 2.411276227891977 request/s
